### Create training set

In [1]:
import numpy as np
from random import shuffle

#add binary string a, and b to create c.
def add(a, b):
    l = max(len(a), len(b));
    a = int(a, 2);
    b = int(b, 2);
    c = bin(a+b)[2:].zfill(l);
    return c;

def str2np(a):
    result = [int(x) for x in a]
    return result;

def lstofstr2tensor(lstofstr):
    tensor = [str2np(string) for string in lstofstr]
    return tensor

def merge_str2list(str1, str2):
    result = [[int(str1[i]), int(str2[i])] for i in xrange(len(str1))]
    return result
N_bit = 15;
#To prevent overloading bit on result, we only generate N_bit-1 string
formatting_string = ('{0:0'+str(N_bit)+'b}')
input_a = [formatting_string.format(i) for i in xrange(2**(N_bit-1))];
input_b = [formatting_string.format(i) for i in xrange(2**(N_bit-1))];
shuffle(input_a);
shuffle(input_b);

N_TOTAL = 2000;

input_a = input_a[:N_TOTAL];
input_b = input_b[:N_TOTAL];
target_c = [add(input_a[i], input_b[i]) for i in xrange(N_TOTAL)];

#Need to reverse the binary string, since we do the calculation from right to left
input_a = [a[::-1] for a in input_a];
input_b = [b[::-1] for b in input_b];
target_c = [c[::-1] for c in target_c];

#Merge string a and b to one input
input_ab = [merge_str2list(input_a[i], input_b[i]) for i in xrange(N_TOTAL)];

target_c = lstofstr2tensor(target_c);

#Convert to numpy array
input_ab = np.asarray(input_ab)
target_c = np.asarray(target_c)

#Split to training and testing dataset
N_TRAIN = 1500;
test_input_ab = input_ab[N_TRAIN:]
test_target_c = target_c[N_TRAIN:] 

train_input_ab = input_ab[:N_TRAIN]
train_target_c = target_c[:N_TRAIN] 


### Use Tensorflow for LSTM
#### First we define the computational graph

In [2]:
import tensorflow as tf
tf.reset_default_graph()

#Placeholder for input and target, [Batch Size, Sequence Length, Input Dimension]
data = tf.placeholder(tf.float32, [None, N_bit, 2])
target = tf.placeholder(tf.float32, [None, N_bit])

num_hidden = 5
time_steps = N_bit

lstmCell = tf.nn.rnn_cell.BasicLSTMCell(num_hidden, forget_bias=1.0, state_is_tuple=True)

#sequence_length 
#unroll the LSTM network
outputs, states = tf.nn.dynamic_rnn(cell=lstmCell, inputs=data, dtype=tf.float32)

# Define weights
out_size = 1;

# Weights for each timestep output, using the same weight
weight = tf.Variable(tf.truncated_normal([num_hidden, out_size], stddev=0.1))
bias = tf.Variable(tf.constant(0.1, shape=[out_size]))

outputs = tf.reshape(outputs, [-1, num_hidden])
prediction = tf.sigmoid(tf.matmul(outputs, weight) + bias)
prediction = tf.reshape(prediction, [-1, time_steps, out_size])
prediction = tf.squeeze(prediction)

cross_entropy = -tf.reduce_mean(target * tf.log(prediction) + (1-target)*tf.log(1-prediction))
#cross_entropy = tf.reduce_mean(cross_entropy)

#cross_entropy = -tf.reduce_sum(target * tf.log(prediction))

optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

final_prediction = prediction>0.5;
mistakes = tf.not_equal(target>0.5, final_prediction) #careful when target is still float32
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))


#### Then we execute the graph

In [3]:
import time
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

BATCH_SIZE = 100
no_of_batches = int(len(train_input_ab)/BATCH_SIZE)
epoch = 200
start = time.time();
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input_ab[ptr:ptr+BATCH_SIZE], train_target_c[ptr:ptr+BATCH_SIZE]
        ptr+=BATCH_SIZE
        sess.run(minimize,{data: inp, target: out})
        
        #print(sess.run(cross_entropy,{data: inp, target: out}))
    print str(time.time()-start), "s Epoch - ", str(i);
incorrect = sess.run(error,{data: test_input_ab, target: test_target_c})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
prediction = sess.run(final_prediction,{data: test_input_ab, target: test_target_c})
prediction.astype(int)

#sess.close()

0.0569939613342 s Epoch -  0
0.103631019592 s Epoch -  1
0.150403022766 s Epoch -  2
0.196510076523 s Epoch -  3
0.243072032928 s Epoch -  4
0.289645910263 s Epoch -  5
0.336657047272 s Epoch -  6
0.382961034775 s Epoch -  7
0.429735898972 s Epoch -  8
0.475718975067 s Epoch -  9
0.523159980774 s Epoch -  10
0.56963801384 s Epoch -  11
0.615669965744 s Epoch -  12
0.662015914917 s Epoch -  13
0.708580970764 s Epoch -  14
0.755018949509 s Epoch -  15
0.801244020462 s Epoch -  16
0.84724187851 s Epoch -  17
0.893456935883 s Epoch -  18
0.939221858978 s Epoch -  19
0.985409975052 s Epoch -  20
1.03158593178 s Epoch -  21
1.0776860714 s Epoch -  22
1.12388586998 s Epoch -  23
1.17010688782 s Epoch -  24
1.21639609337 s Epoch -  25
1.26229190826 s Epoch -  26
1.30831503868 s Epoch -  27
1.35465192795 s Epoch -  28
1.40111088753 s Epoch -  29
1.44819498062 s Epoch -  30
1.49461507797 s Epoch -  31
1.54043102264 s Epoch -  32
1.58660387993 s Epoch -  33
1.63269090652 s Epoch -  34
1.679429054

KeyboardInterrupt: 

In [103]:
#Some prediction examples:
prediction = sess.run(final_prediction,{data: test_input_ab, target: test_target_c})
print test_input_ab[59]
print test_target_c[59]
print prediction[59]

[[0 0]
 [0 1]
 [0 1]
 [1 1]
 [0 1]
 [0 1]
 [1 1]
 [0 1]
 [1 0]
 [1 0]
 [0 0]
 [0 0]
 [0 1]
 [1 1]
 [0 0]]
[0 1 1 0 0 0 1 0 0 0 1 0 1 0 1]
[False  True  True False False False  True False False False  True False
  True False  True]
